# Notebook 04: Structured Outputs & JSON Schema

**Objectives:**
- Extract structured data with json_extract.v1 prompt
- Validate against JSON schemas
- Repair malformed JSON
- Log token costs for validation + repair cycles

In [ ]:
import sys
import pprint
sys.path.append('..')

from utils.prompts import render
from utils.llm_client import LLMClient
from utils.logging_utils import log_llm_call
from utils.router import pick_model
from utils.json_utils import safe_parse_json, validate_json_schema, create_simple_schema, format_schema_for_prompt
import json

## Part 1: JSON Extraction with Schema

Define a schema and extract structured data.

In [ ]:
text = """The CloudSync Pro Business plan costs 20 LKR per user per month.
It includes 10TB storage and is currently available."""

schema = create_simple_schema({
                            "name": "string",
                            "price": "number",
                            "currency": "string",
                            "in_stock": "boolean"
                        }, required=["name", "price", "currency"])
pprint.pprint(schema)